In [4]:
import typing

import pandas as pd

import config

import logging
import tqdm
import torch
import numpy as np
import requests


/Users/SjoerdStolwijk/anaconda3/envs/DefiningDebateQuality/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


ModuleNotFoundError: No module named 'plotly'

In [ ]:
CFG = config.Config()

In [ ]:
sentences = ["I'm going to the bank.", "I'm going to the bank.", "I'm going to the bank.", "I need to visit the bank today.", "I'm heading to the bank to withdraw some cash.", "The bank is where I'm off to.", "I have an appointment at the bank.", "Let's spend the day by the river bank.", "The river bank is a peaceful place to relax.", "I enjoy walking along the river bank.", "We can have a picnic by the river bank.",
"She has a cool job.", "She has a cool job.", "She has a cool job.", "Her job is really interesting and fun.", "She works in a creative field and loves it.", "That job of hers is so unique.", "She's lucky to have such a cool profession.", "Her workplace is always chilly.", "She works in a refrigerated environment.", "The temperature in her office is freezing.", "She needs to bundle up for her job.",
"You need to check the trunk.", "You need to check the trunk.", "You need to check the trunk.", "Don't forget to look in the trunk of the car.", "There might be something important in the trunk.", "Make sure to verify the contents of the trunk.", "The trunk needs to be inspected for any damage.", "Take a look at the tree trunk for any damage.", "The trunk of the old tree might have some interesting carvings.", "Check if the tree trunk needs to be treated for pests.", "See if there are any unique patterns or textures on the tree trunk.",
"I saw her duck.", "I saw her duck.", "I saw her duck.", "Her duck was waddling in the park.", "The duck she owns is so cute.", "I spotted her duck by the pond.", "Her duck was quacking loudly.", "She ducked to avoid the flying object.", "Her quick ducking saved her from the falling branch.", "I noticed her sudden ducking movement.", "She ducked and dodged the incoming ball.",
"She has a great figure.", "She has a great figure.", "She has a great figure.", "Her body shape is very flattering.", "She carries herself with grace and confidence.", "Her figure is well-proportioned and attractive.", "She knows how to dress to highlight her figure.", "The numerical figure she presented was impressive.", "Her calculations yielded a significant figure.", "The data supports a substantial figure.", "The figure she quoted was accurate and reliable."]

context = ["bank", "bank-money", "bank-river", "bank-money", "bank-money", "bank-money", "bank-money", "bank-river", "bank-river", "bank-river", "bank-river", "cool", "cool-nice", "cool-cold", "cool-nice", "cool-nice", "cool-nice", "cool-nice", "cool-cold", "cool-cold", "cool-cold", "cool-cold", "trunk", "trunk-car", "trunk-tree", "trunk-car", "trunk-car", "trunk-car", "trunk-car", "trunk-tree", "trunk-tree", "trunk-tree", "trunk-tree", 
           "duck", "duck-animal", "duck-down", "duck-animal", "duck-animal", "duck-animal", "duck-animal", "duck-down", "duck-down", "duck-down", "duck-down", "figure", "figure-body", "figure-number", "figure-body", "figure-body", "figure-body", "figure-body", "figure-number", "figure-number", "figure-number", "figure-number"]

set = ["bank-base", "bank-base-money", "bank-base-river", "bank-money", "bank-money", "bank-money", "bank-money", "bank-river", "bank-river", "bank-river", "bank-river", "cool-base", "cool-base-nice", "cool-base-cold", "cool-nice", "cool-nice", "cool-nice", "cool-nice", "cool-cold", "cool-cold", "cool-cold", "cool-cold", "trunk-base", "trunk-base-car", "trunk-base-tree", "trunk-car", "trunk-car", "trunk-car", "trunk-car", "trunk-tree", "trunk-tree", "trunk-tree", "trunk-tree", 
           "duck-base", "duck-base-animal", "duck-base-down", "duck-animal", "duck-animal", "duck-animal", "duck-animal", "duck-down", "duck-down", "duck-down", "duck-down", "figure-base", "figure-base-body", "figure-base-number", "figure-body", "figure-body", "figure-body", "figure-body", "figure-number", "figure-number", "figure-number", "figure-number"]

dataset = pd.DataFrame({'Sentence': sentences, 'Context': context, 'Set': set})

In [ ]:
dataset.head()

,Sentence,Context,Set
0,I'm going to the bank.,bank,bank-base
1,I'm going to the bank.,bank-money,bank-base-money
2,I'm going to the bank.,bank-river,bank-base-river
3,I need to visit the bank today.,bank-money,bank-money
4,I'm heading to the bank to withdraw some cash.,bank-money,bank-money


In [ ]:
MODEL: str = "llama3:70b-instruct-q6_K" # "mixtral:8x7b-instruct-v0.1-q6_K"


dict_keys(['political_ideology', 'embedding_semantic_rich', 'political_negativity', 'embedding_base', 'argument'])

In [2]:
templates = [
    {"name": "embed_base", "template": 'You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = '},
    {"name": "embed_rich", "template": 'You help me get embeddings for a sentence that will help me to determine semantic sentence similarity. I provide you a sentence and you reply only with that exact sentence. Context = '},
]

template_dict = pd.DataFrame(templates).to_dict(orient='records')

In [3]:
embed_base: typing.Dict[str, np.ndarray] = {}
embed_rich: typing.Dict[str, np.ndarray] = {}

In [9]:
for item in template_dict:
    name = item['name']
    template = item['template']
    for index, row in tqdm.tqdm(dataset.iterrows(), total=len(dataset)):
        context = row["Context"]
        sentence = row["Sentence"]
        try: 
            embed = np.array(requests.post(
                'https://inf.cl.uni-trier.de/embed/',
                json={'model': MODEL, 'prompt': template + context + '; Sentence: ' + sentence}
                ).json()["response"])
        except Exception as _e:
            logging.warning(_e)
            embed = None
        
        if name=='embed_base':
            embed_base[index] = embed
        if name=='embed_rich':
            embed_rich[index] = embed

       

100%|██████████| 55/55 [00:13<00:00,  4.18it/s]


In [10]:
dataset_w_embeds = dataset.join(pd.Series(embed_base, name="embed_base")).join(pd.Series(embed_rich, name="embed_rich"))
#dataset_w_embeds.to_parquet(f'{CFG.report_dir}/dataset.embeds.parquet')
dataset_w_embeds.head()

,Sentence,Context,Set,embed_base,embed_rich
0,I'm going to the bank.,bank,bank-base,"[0.2825756371021271, -0.7305834889411926, -0.6...","[0.40476152300834656, -0.40213334560394287, -0..."
1,I'm going to the bank.,bank-money,bank-base-money,"[0.4423738121986389, -0.6933848857879639, -0.6...","[0.4015294909477234, -0.24794012308120728, -0...."
2,I'm going to the bank.,bank-river,bank-base-river,"[0.31343695521354675, -0.6396633982658386, -0....","[0.3487618863582611, -0.11420242488384247, -0...."
3,I need to visit the bank today.,bank-money,bank-money,"[0.4185699224472046, -0.8129321336746216, -0.7...","[0.3717159330844879, -0.28542259335517883, -0...."
4,I'm heading to the bank to withdraw some cash.,bank-money,bank-money,"[0.35524073243141174, -0.5443449020385742, -0....","[0.322308748960495, -0.14055457711219788, -0.3..."


In [ ]:
#add visualisation:


In [113]:
#comparing effect of prompt (base versus rich) and context (base versus context) for distance base-river: base-prompt results
results: typing.Dict[typing.Tuple[str, str], float] = {}
groups = {
    "prombase-senbase": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base"]["embed_base"],
    "prombase-senriver": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-river"]["embed_base"],
    "prombase-senbase-river": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-river"]["embed_base"]
}

In [115]:
#comparing effect of prompt (base versus rich) and context (base versus context) for distance base-river: rich-prompt results
groups = {
    "promrich-senbase": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base"]["embed_rich"],
    "promrich-senriver": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-river"]["embed_rich"],
    "promrich-senbase-river": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-river"]["embed_rich"]
}

In [116]:
dist = torch.nn.PairwiseDistance()

for label_1, c_1 in groups.items():
    for label_2, c_2 in groups.items():

        if (
            (label_1, label_2) in results.keys() or 
            (label_2, label_1) in results.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        results[(label_1, label_2)] = res

        print(f'{label_1}:{label_2}:{res.item()}')

promrich-senbase:promrich-senbase:3.1999999999999965e-05
promrich-senbase:promrich-senriver:4.316801094975404
promrich-senbase:promrich-senbase-river:3.6447948132129264
promrich-senriver:promrich-senriver:1.6584892020724247
promrich-senriver:promrich-senbase-river:2.08751125735614
promrich-senbase-river:promrich-senbase-river:3.1999999999999965e-05


In [117]:
results

{('prombase-senbase',
  'prombase-senbase'): tensor(3.2000e-05, dtype=torch.float64),
 ('prombase-senbase',
  'prombase-senriver'): tensor(5.6014, dtype=torch.float64),
 ('prombase-senbase',
  'prombase-senbase-river'): tensor(4.7680, dtype=torch.float64),
 ('prombase-senriver',
  'prombase-senriver'): tensor(2.1001, dtype=torch.float64),
 ('prombase-senriver',
  'prombase-senbase-river'): tensor(2.5120, dtype=torch.float64),
 ('prombase-senbase-river',
  'prombase-senbase-river'): tensor(3.2000e-05, dtype=torch.float64),
 ('promrich-senbase',
  'promrich-senbase'): tensor(3.2000e-05, dtype=torch.float64),
 ('promrich-senbase',
  'promrich-senriver'): tensor(4.3168, dtype=torch.float64),
 ('promrich-senbase',
  'promrich-senbase-river'): tensor(3.6448, dtype=torch.float64),
 ('promrich-senriver',
  'promrich-senriver'): tensor(1.6585, dtype=torch.float64),
 ('promrich-senriver',
  'promrich-senbase-river'): tensor(2.0875, dtype=torch.float64),
 ('promrich-senbase-river',
  'promrich-se

In line with expectations:
it appears distances for base prompts between base sentences and river-sentences are smaller when context is prompted (2.5) than when it is not specified (5.6). 
For rich prompts distances between base sentences and river-sentences are smaller when context is prompted (2.1) than when it is not specified (4.3). 
Distances from base sentence to river sentences is smaller for rich prompts than base prompts
Difference between base and river sentences is smaller for base  (4.3168-3.6448 = 2.23) than rich (5.6014-2.5120= 3.09) prompts -> giving context to base sentence has a smaller effect on the base prompt than the rich prompt results 

How about the money sentences?

In [123]:
#comparing effect of prompt (base versus rich) and context (base versus context) for distance base-money: base-prompt results
results2: typing.Dict[typing.Tuple[str, str], float] = {}
groups = {
    "prombase-senbase": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base"]["embed_base"],
    "prombase-senmoney": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-money"]["embed_base"],
    "prombase-senbase-money": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-money"]["embed_base"]
}

In [125]:
#comparing effect of prompt (base versus rich) and context (base versus context) for distance base-money: rich-prompt results
groups = {
    "promrich-senbase": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base"]["embed_rich"],
    "promrich-senmoney": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-money"]["embed_rich"],
    "promrich-senbase-money": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-money"]["embed_rich"]
}

In [126]:
dist = torch.nn.PairwiseDistance()

for label_1, c_1 in groups.items():
    for label_2, c_2 in groups.items():

        if (
            (label_1, label_2) in results2.keys() or 
            (label_2, label_1) in results2.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        results2[(label_1, label_2)] = res

        print(f'{label_1}:{label_2}:{res.item()}')

promrich-senbase:promrich-senbase:3.1999999999999965e-05
promrich-senbase:promrich-senmoney:2.871530877718972
promrich-senbase:promrich-senbase-money:1.8333785117074113
promrich-senmoney:promrich-senmoney:1.8499498945289596
promrich-senmoney:promrich-senbase-money:2.115557370752927
promrich-senbase-money:promrich-senbase-money:3.1999999999999965e-05


In [127]:
results2

{('prombase-senbase',
  'prombase-senbase'): tensor(3.2000e-05, dtype=torch.float64),
 ('prombase-senbase',
  'prombase-senmoney'): tensor(3.4596, dtype=torch.float64),
 ('prombase-senbase',
  'prombase-senbase-money'): tensor(2.9066, dtype=torch.float64),
 ('prombase-senmoney',
  'prombase-senmoney'): tensor(2.1642, dtype=torch.float64),
 ('prombase-senmoney',
  'prombase-senbase-money'): tensor(2.4597, dtype=torch.float64),
 ('prombase-senbase-money',
  'prombase-senbase-money'): tensor(3.2000e-05, dtype=torch.float64),
 ('promrich-senbase',
  'promrich-senbase'): tensor(3.2000e-05, dtype=torch.float64),
 ('promrich-senbase',
  'promrich-senmoney'): tensor(2.8715, dtype=torch.float64),
 ('promrich-senbase',
  'promrich-senbase-money'): tensor(1.8334, dtype=torch.float64),
 ('promrich-senmoney',
  'promrich-senmoney'): tensor(1.8499, dtype=torch.float64),
 ('promrich-senmoney',
  'promrich-senbase-money'): tensor(2.1156, dtype=torch.float64),
 ('promrich-senbase-money',
  'promrich-se

In line with expectations:
it appears distances for base prompts between base sentences and money-sentences are smaller when context is prompted (2.5) than when it is not specified (3.5). 
For rich prompts distances between base sentences and money-sentences are smaller when context is prompted (2.1) than when it is not specified (2.9). 
Distances from base sentence to money sentences is smaller for rich prompts than base prompts
**However** :Difference between base and money sentences with and without context is larger for base  (3.4596-2.4597 = 1.00) than rich (2.8715-2.1156= 0.76) prompts -> giving context to base sentence has a larger effect on the base prompt than the rich prompt results 

summarizing: giving context improves embedding-interpretation, we need further testing to verify whether rich prompting outperforms base prompting

perhaps we could compare whether the difference between the bank-base sentence in a river context and the bank-base sentence in a money context is larger for rich than base prompts? Similarly the distance between river and money sentences should be larger for rich prompts

Reasoning: rich prompts should be more sensitive to meaning, so changing the meaning of the base sentence should have a larger influence on embeddings of rich prompts

In [136]:
#comparing effect of prompt (base versus rich) for river versus money sentences: base-sentence results
results3: typing.Dict[typing.Tuple[str, str], float] = {}
groups = {
    "prombase-senbase-river": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-river"]["embed_base"],
    "prombase-senbase-money": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-money"]["embed_base"],
    "promrich-senbase-river": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-river"]["embed_rich"],
    "promrich-senbase-money": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-base-money"]["embed_rich"]
    }

dist = torch.nn.PairwiseDistance()

for label_1, c_1 in groups.items():
    for label_2, c_2 in groups.items():

        if (
            (label_1, label_2) in results3.keys() or 
            (label_2, label_1) in results3.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        results3[(label_1, label_2)] = res

        print(f'{label_1}:{label_2}:{res.item()}')

prombase-senbase-river:prombase-senbase-river:3.1999999999999965e-05
prombase-senbase-river:prombase-senbase-money:4.800173477526052
prombase-senbase-river:promrich-senbase-river:7.295951348317239
prombase-senbase-river:promrich-senbase-money:8.15563421229664
prombase-senbase-money:prombase-senbase-money:3.1999999999999965e-05
prombase-senbase-money:promrich-senbase-river:8.051872238071171
prombase-senbase-money:promrich-senbase-money:7.145259191029507
promrich-senbase-river:promrich-senbase-river:3.1999999999999965e-05
promrich-senbase-river:promrich-senbase-money:3.4803490395104006
promrich-senbase-money:promrich-senbase-money:3.1999999999999965e-05


Against expectations:
it appears distances for base prompts between base-river sentences and base-money-sentences (4.8) are larger than for rich prompts (3.5)

In [137]:
#comparing effect of prompt (base versus rich) for river versus money sentences: base-sentence results
results4: typing.Dict[typing.Tuple[str, str], float] = {}
groups = {
    "prombase-senriver": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-river"]["embed_base"],
    "prombase-senmoney": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-money"]["embed_base"],
    "promrich-senriver": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-river"]["embed_rich"],
    "promrich-senmoney": dataset_w_embeds[dataset_w_embeds["Set"] == "bank-money"]["embed_rich"]
    }

dist = torch.nn.PairwiseDistance()

for label_1, c_1 in groups.items():
    for label_2, c_2 in groups.items():

        if (
            (label_1, label_2) in results4.keys() or 
            (label_2, label_1) in results4.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        results4[(label_1, label_2)] = res

        print(f'{label_1}:{label_2}:{res.item()}')

prombase-senriver:prombase-senriver:2.10008842155786
prombase-senriver:prombase-senmoney:5.755950055125333
prombase-senriver:promrich-senriver:7.14581244176149
prombase-senriver:promrich-senmoney:8.552889429688943
prombase-senmoney:prombase-senmoney:2.1641586372910844
prombase-senmoney:promrich-senriver:8.144890753162407
prombase-senmoney:promrich-senmoney:7.229541950952813
promrich-senriver:promrich-senriver:1.6584892020724247
promrich-senriver:promrich-senmoney:4.583495704374738
promrich-senmoney:promrich-senmoney:1.8499498945289596


Against expectations:
it appears distances for base prompts between river sentences and money-sentences (5.7) are larger than for rich prompts (4.58)

Also it appears prompts have a strong influence on subsequent embeddings since the difference among river sentences within the base prompt (2.1) and the rich prompt (1.66) are rather small, while the differences among river sentences across prompts are very large (7.1), even though the difference between prompts ("that will help me to determine semantic sentence similarity") appears arbitrary to this difference among river sentences (which thus are rather similar when compared for the same prompt using the same context).

Summarizing: it appears that the base prompt in combination with context specification yield best results

In [18]:
#does prombase do a good job to differentiate across all sets:

dataset_w_embeds[~dataset_w_embeds['Set'].str.contains('base')].head()


,Sentence,Context,Set,embed_base,embed_rich
3,I need to visit the bank today.,bank-money,bank-money,"[0.4185699224472046, -0.8129321336746216, -0.7...","[0.3717159330844879, -0.28542259335517883, -0...."
4,I'm heading to the bank to withdraw some cash.,bank-money,bank-money,"[0.35524073243141174, -0.5443449020385742, -0....","[0.322308748960495, -0.14055457711219788, -0.3..."
5,The bank is where I'm off to.,bank-money,bank-money,"[0.4175596833229065, -0.5876224637031555, -0.6...","[0.36472979187965393, -0.1458236128091812, -0...."
6,I have an appointment at the bank.,bank-money,bank-money,"[0.3939899802207947, -0.7644816040992737, -0.7...","[0.2895117998123169, -0.27759850025177, -0.443..."
7,Let's spend the day by the river bank.,bank-river,bank-river,"[0.30943161249160767, -0.5281971096992493, -0....","[0.33497363328933716, -0.12277282774448395, -0..."


In [19]:
#groupby sets, excluding base-category
setgrouped = dataset_w_embeds[~dataset_w_embeds['Set'].str.contains('base')].groupby("Set")["embed_base"]
setgrouped.head()

3     [0.4185699224472046, -0.8129321336746216, -0.7...
4     [0.35524073243141174, -0.5443449020385742, -0....
5     [0.4175596833229065, -0.5876224637031555, -0.6...
6     [0.3939899802207947, -0.7644816040992737, -0.7...
7     [0.30943161249160767, -0.5281971096992493, -0....
8     [0.3739273250102997, -0.6068928837776184, -0.6...
9     [0.37967318296432495, -0.7677608728408813, -0....
10    [0.2575017213821411, -0.6640542149543762, -0.7...
14    [0.36158478260040283, -0.7932509779930115, -0....
15    [0.4043620526790619, -0.8143025040626526, -0.6...
16    [0.4671706259250641, -0.78300940990448, -0.611...
17    [0.3373933434486389, -0.7002600431442261, -0.6...
18    [0.38493454456329346, -0.5809698104858398, -0....
19    [0.3893470764160156, -0.5666419267654419, -0.4...
20    [0.40476319193840027, -0.5463060140609741, -0....
21    [0.4274798333644867, -0.7088919281959534, -0.5...
25    [0.2471407949924469, -0.3737742006778717, -0.6...
26    [0.4128078818321228, -0.46863067150115967,

In [20]:
dist = torch.nn.PairwiseDistance()
results5: typing.Dict[typing.Tuple[str, str], float] = {}
for label_1, c_1 in setgrouped:
    for label_2, c_2 in setgrouped:

        if (
            (label_1, label_2) in results5.keys() or 
            (label_2, label_1) in results5.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        results5[(label_1, label_2)] = res

        print(f'{label_1}:{label_2}:{res.item()}')

bank-money:bank-money:2.1641586372910844
bank-money:bank-river:5.755950008254665
bank-money:cool-cold:8.107997314331437
bank-money:cool-nice:7.233491794700285
bank-money:duck-animal:7.828895944417515
bank-money:duck-down:9.404484968721276
bank-money:figure-body:8.111505554415858
bank-money:figure-number:8.060565296128399
bank-money:trunk-car:7.007210867748952
bank-money:trunk-tree:7.330297362712321
bank-river:bank-river:2.10008842155786
bank-river:cool-cold:7.8804507797566
bank-river:cool-nice:7.115697953592645
bank-river:duck-animal:7.1615193942651905
bank-river:duck-down:9.008679758531105
bank-river:figure-body:7.969254843776894
bank-river:figure-number:7.415517898698991
bank-river:trunk-car:6.851839628994174
bank-river:trunk-tree:6.324327350235041
cool-cold:cool-cold:2.111790495816015
cool-cold:cool-nice:5.138850784483196
cool-cold:duck-animal:7.736483663274887
cool-cold:duck-down:8.572636189336837
cool-cold:figure-body:7.933266086605007
cool-cold:figure-number:7.774803675916708
coo

In [24]:

for i in range(3, len(results5), 4):
    print(list(results5.items())[i])

(('bank-money', 'cool-nice'), tensor(7.2335, dtype=torch.float64))
(('bank-money', 'figure-number'), tensor(8.0606, dtype=torch.float64))
(('bank-river', 'cool-cold'), tensor(7.8805, dtype=torch.float64))
(('bank-river', 'figure-body'), tensor(7.9693, dtype=torch.float64))
(('cool-cold', 'cool-cold'), tensor(2.1118, dtype=torch.float64))
(('cool-cold', 'figure-body'), tensor(7.9333, dtype=torch.float64))
(('cool-nice', 'cool-nice'), tensor(2.2760, dtype=torch.float64))
(('cool-nice', 'figure-number'), tensor(7.0513, dtype=torch.float64))
(('duck-animal', 'duck-down'), tensor(5.7525, dtype=torch.float64))
(('duck-animal', 'trunk-tree'), tensor(7.5812, dtype=torch.float64))
(('duck-down', 'trunk-car'), tensor(8.5960, dtype=torch.float64))
(('figure-body', 'trunk-car'), tensor(7.7876, dtype=torch.float64))
(('figure-number', 'trunk-tree'), tensor(7.5752, dtype=torch.float64))


appears to work quite well

Some validity checks:

In [94]:
#maybe look at the output that is embedded, did the output yield correctly or are there traces of the prompt left in the output?

output_base: typing.Dict[str, np.ndarray] = {}
output_rich: typing.Dict[str, np.ndarray] = {}

for item in template_dict:
    name = item['name']
    template = item['template']
    for index, row in tqdm.tqdm(dataset.iterrows(), total=len(dataset)):
        context = row["Context"]
        sentence = row["Sentence"]
        try: 
            output = np.array(requests.post(
                'https://inf.cl.uni-trier.de/',
                json={'model': MODEL, 'prompt': template + context + '; Sentence: ' + sentence}
                ).json()["response"])
        except Exception as _e:
            logging.warning(_e)
            output = None
        
        if name=='embed_base':
            output_base[index] = output
        if name=='embed_rich':
            output_rich[index] = output


100%|██████████| 55/55 [01:07<00:00,  1.23s/it]


In [138]:
for item in template_dict:
    name = item['name']
    template = item['template']
    for index, row in tqdm.tqdm(dataset[:10].iterrows(), total=len(dataset)):
        context = row["Context"]
        sentence = row["Sentence"]
        print(template + context + '; Sentence = ' + sentence)

 18%|█▊        | 10/55 [00:00<00:00, 5127.51it/s]


You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = bank; Sentence = I'm going to the bank.
You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = bank-money; Sentence = I'm going to the bank.
You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = bank-river; Sentence = I'm going to the bank.
You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = bank-money; Sentence = I need to visit the bank today.
You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = bank-money; Sentence = I'm heading to the bank to withdraw some cash.
You help me get embeddi

 18%|█▊        | 10/55 [00:00<00:00, 9416.94it/s]

You help me get embeddings for a sentence that will help me to determine semantic sentence similarity. I provide you a sentence and you reply only with that exact sentence. Context = bank; Sentence = I'm going to the bank.
You help me get embeddings for a sentence that will help me to determine semantic sentence similarity. I provide you a sentence and you reply only with that exact sentence. Context = bank-money; Sentence = I'm going to the bank.
You help me get embeddings for a sentence that will help me to determine semantic sentence similarity. I provide you a sentence and you reply only with that exact sentence. Context = bank-river; Sentence = I'm going to the bank.
You help me get embeddings for a sentence that will help me to determine semantic sentence similarity. I provide you a sentence and you reply only with that exact sentence. Context = bank-money; Sentence = I need to visit the bank today.
You help me get embeddings for a sentence that will help me to determine semantic

In [139]:
output_base

{0: array("I'm going to the bank.", dtype='<U22'),
 1: array("I'm going to the bank.", dtype='<U22'),
 2: array("I'm going to the bank.", dtype='<U22'),
 3: array('I need to visit the bank today.', dtype='<U31'),
 4: array("I'm heading to the bank to withdraw some cash.", dtype='<U46'),
 5: array("The bank is where I'm off to.", dtype='<U29'),
 6: array('I have an appointment at the bank.', dtype='<U34'),
 7: array("Let's spend the day by the river bank.", dtype='<U38'),
 8: array('The river bank is a peaceful place to relax.', dtype='<U44'),
 9: array('I enjoy walking along the river bank.', dtype='<U37'),
 10: array('We can have a picnic by the river bank.', dtype='<U39'),
 11: array('She has a cool job.', dtype='<U19'),
 12: array('She has a cool job.', dtype='<U19'),
 13: array('She has a cool job.', dtype='<U19'),
 14: array('Her job is really interesting and fun.', dtype='<U38'),
 15: array('She works in a creative field and loves it.', dtype='<U43'),
 16: array('That job of hers

In [140]:
output_rich

{0: array("I'm going to the bank.", dtype='<U22'),
 1: array("I'm going to the bank.", dtype='<U22'),
 2: array("I'm going to the bank.", dtype='<U22'),
 3: array('I need to visit the bank today.', dtype='<U31'),
 4: array("I'm heading to the bank to withdraw some cash.", dtype='<U46'),
 5: array("The bank is where I'm off to.", dtype='<U29'),
 6: array('I have an appointment at the bank.', dtype='<U34'),
 7: array("Let's spend the day by the river bank.", dtype='<U38'),
 8: array('The river bank is a peaceful place to relax.', dtype='<U44'),
 9: array('I enjoy walking along the river bank.', dtype='<U37'),
 10: array('We can have a picnic by the river bank.', dtype='<U39'),
 11: array('She has a cool job.', dtype='<U19'),
 12: array('She has a nice job.', dtype='<U19'),
 13: array('She has a cool job.', dtype='<U19'),
 14: array('Her job is really interesting and fun.', dtype='<U38'),
 15: array('She works in a creative field and loves it.', dtype='<U43'),
 16: array('That job of hers